In [1]:
import pandas as pd

In [4]:
df=pd.read_csv('/content/HR_comma_sep.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14999 entries, 0 to 14998
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   satisfaction_level     14999 non-null  float64
 1   last_evaluation        14999 non-null  float64
 2   number_project         14999 non-null  int64  
 3   average_montly_hours   14999 non-null  int64  
 4   time_spend_company     14999 non-null  int64  
 5   Work_accident          14999 non-null  int64  
 6   left                   14999 non-null  int64  
 7   promotion_last_5years  14999 non-null  int64  
 8   Department             14999 non-null  object 
 9   salary                 14999 non-null  object 
dtypes: float64(2), int64(6), object(2)
memory usage: 1.1+ MB


In [6]:
df.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000,14999.000000
mean,0.612834,0.716102,3.803054,201.050337,3.498233,0.144610,0.238083,0.021268
std,0.248631,0.171169,1.232592,49.943099,1.460136,0.351719,0.425924,0.144281
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [7]:
# Check for missing values
print(df.isnull().sum())


satisfaction_level       0
last_evaluation          0
number_project           0
average_montly_hours     0
time_spend_company       0
Work_accident            0
left                     0
promotion_last_5years    0
Department               0
salary                   0
dtype: int64


In [11]:
# Convert categorical variables to numerical using one-hot encoding
df_encoded = pd.get_dummies(df, columns=['Department', 'salary'], drop_first=True)

# Convert a specific column to integers (e.g., 'satisfaction_level')
df_encoded['satisfaction_level'] = df_encoded['satisfaction_level'].astype(int)

# Display the first few rows
print(df_encoded.head())

   satisfaction_level  last_evaluation  number_project  average_montly_hours  \
0                   0             0.53               2                   157   
1                   0             0.86               5                   262   
2                   0             0.88               7                   272   
3                   0             0.87               5                   223   
4                   0             0.52               2                   159   

   time_spend_company  Work_accident  left  promotion_last_5years  \
0                   3              0     1                      0   
1                   6              0     1                      0   
2                   4              0     1                      0   
3                   5              0     1                      0   
4                   3              0     1                      0   

   Department_RandD  Department_accounting  Department_hr  \
0             False                  False 

In [12]:
# Define X (features) and y (target)
X = df_encoded.drop(columns=['left'])
y = df_encoded['left']


In [13]:
from sklearn.model_selection import train_test_split

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print dataset sizes
print(f"Training data: {X_train.shape}, Testing data: {X_test.shape}")


Training data: (11999, 18), Testing data: (3000, 18)


In [14]:
from sklearn.preprocessing import StandardScaler

# Initialize scaler
scaler = StandardScaler()

# Scale training and testing data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [30]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the logistic regression model
model = LogisticRegression()
model.fit(X_train_scaled, y_train)


LogisticRegression()

In [31]:
# Make predictions on test data
y_pred = model.predict(X_test_scaled)


In [32]:
model.score(X_test_scaled,y_test)

0.7506666666666667

In [33]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=3, interaction_only=True)  # Using cubic interactions
X_train_poly3 = poly.fit_transform(X_train_scaled)
X_test_poly3 = poly.transform(X_test_scaled)

# Train with higher-degree polynomial features
model.fit(X_train_poly3, y_train)
y_pred = model.predict(X_test_poly3)

accuracy = accuracy_score(y_test, y_pred)
print(f"Final Accuracy with Degree 3 Polynomial Features: {accuracy:.4f}")


Final Accuracy with Degree 3 Polynomial Features: 0.9273


In [40]:
# Drop less relevant features before polynomial transformation
X_train_reduced = pd.DataFrame(X_train_scaled, columns=X_train.columns).drop(['Work_accident', 'promotion_last_5years'], axis=1)
X_test_reduced = pd.DataFrame(X_test_scaled, columns=X_test.columns).drop(['Work_accident', 'promotion_last_5years'], axis=1)

# Apply Polynomial Transformation
poly = PolynomialFeatures(degree=2, interaction_only=True)
X_train_poly_reduced = poly.fit_transform(X_train_reduced)
X_test_poly_reduced = poly.transform(X_test_reduced)

# Train and Evaluate
model.fit(X_train_poly_reduced, y_train)
y_pred = model.predict(X_test_poly_reduced)
model.score(X_test_poly_reduced, y_test)
print(f"Accuracy After Feature Reduction & Polynomial Features: {model.score(X_test_poly_reduced, y_test):.4f}")

Accuracy After Feature Reduction & Polynomial Features: 0.9210
